In [5]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Modelling learning systems in artificial and spiking neural networks

</br></br>

* Author: Jens Egholm Pedersen ``<xtp778@alumni.ku.dk>``
* Supervisor: Martin Elsman ``<mael@diku.dk>``

## Presentation agenda

1. Introduction
2. Coding information in spiking neural networks
3. Domain-specific language: Volr
4. Classification experiments
5. Findings and reflections

## 1. Introduction 

1. Motivation 
2. Description
3. Hypotheses

### 1.1 Motivation 

Spiking neural networks (SNN) promising, but
* Diverse and non-standard representations
* Not much attention to information representation
* Spill-over from artificial neural networks (ANN)

### 1.2 Description

* Theoretical
  * Investigated coding for SNN
  * Backpropagation mechanisms for SNN
  * DSL specification
* Practical
  * Implemented domain-specific language (DSL)
  * Implemented machine learning for ANN and SNN
  * Performed experiments


### 1.3 Hypotheses

1. The Volr DSL can translate into spiking and non-spiking neural networks such that the network topologies are retained.
2. Using training it is possible for spiking and non-spiking models to solve an MNIST recognition task.


## 2. Neural networks
  1. ANN: feedforward and backpropagation
  2. SNN: electrical model
  2. Coding scheme for SNN

### 2.1 ANN: Feedforward

* Computational graph structures

<img src="ff.png" style="width:48%; float:left"/>
<img src="bp.png" style="width:48%; float:right"/>


<span style="font-size: 70%">Source: R. Rojas: Neural Networks, Springer 1996</span>

### 2.1 ANN: Backpropagation

* Backpropagation $\delta$, given an activation function $\sigma$, weights $w$ and two neuron groups $j$ and $k$:
\begin{equation}
  \delta_j = \sigma'_j \sum_k w_{kj} \delta_k 
\end{equation}

### 2.1 ANN: Parallel composition

* Parallel computational structures

<img src="ff2.png" style="width:48%; float:left"/>
<img src="bp2.png" style="width:48%; float:right"/>

<span style="font-size: 70%">Source: R. Rojas: Neural Networks, Springer 1996</span>

### 2.2 Electrical model of a leaky-integrate and fire (LIF) neuron

<table>
    <tr style="background: white">
        <td>$\frac{dv}{dt} = - {1 \over \tau_{c}} (V - J R)$</td>
        <td><img src="rc.png" style="width: 66%; float: right"/></td>
    </tr>
</table>

Where $V$ is the voltage difference between the inner and outer neuron membranes, $J_M$ is the input charge, $R$ is the leak resistance and $\tau_{C}$ is the membrane time constant

$J_R = {V \over R}$

<center><img src="lif.png" style="width:40%"/></center>

### 2.2 LIF activations and parameters

<p style="font-size:70%">Input: Constant input current of 2 nA</p>
<img src="../report/images/membrane.png" style="width: 50%; float:left;"/>

<img src="parameters.png" style="width: 48%; float: right;"/>

### 2.3 Coding scheme for SNN

Given some activation potential $x$, input current $v$, voltage threshold $V_{thr}$, a max spike rate $r_{max}$, the neuron membrane potential $v$, and some error $\epsilon$, the neuron spike rate over an interval $t$ is approximately:

\begin{equation}
r_i^1(t) = x_i^1 r_{max} {V_{thr} \over V_{thr} + \epsilon_i^1 } - {v_i^1(t) \over t (V_{thr} + \epsilon_i^1) }
\end{equation}


<span style="font-size: 70%">Source: Rueckauer et al., Frontiers in Neuroscience 11:682, 2017.</span>

### 2.3 Translation from current to spike rate

<img src="../report/images/spike_rate.png" style="width:50%"/>

### 2.3 Composition of  populations

$w^l = {0.065 \over n^{l-1}}, r \in [0;38]$

![](../report/images/spike_rate_mnist.png)

## 3. Volr: DSL for neural networks
  1. Requirements
  2. Implementation

### 3.1 DSL requirements
1. Semantic consistency
  * Between ANN and SNN
2. Translation to ANN and SNN
3. Learning
  * Backpropagation
4. Well-typed
  * Type checking at compile time

### 3.2 Implementation

<img src="architecture.svg" style="width:35%; float:left;"/>

<table style="font-size:100%; width:65%">
    <tr><td colspan=2><center>
  <code><b>dense</b> 2 4 ⦶ <b>dense</b> 4 2</code></center></td></tr>
    <tr>
        <td width="70%" style="font-size:60%; vertical-align: top; width=70%;">
<pre style="padding:0">...
let x0 = dense (2, 4)
let x1 = dense (4, 2)
let n = connect_layers x0 x1
...
</pre>
</td>
<td style="font-size:60%">
<pre style="padding:0">...
p1 = Population(2, ...)
p3 = Population(4, ...)
p5 = Population(2, ...)
layer0 = Dense(p1, p3)
layer1 = Dense(p3, p5)
...
</pre>
</td>
    </tr>
</table>

### 3.2 Implementation: Parallel

<table style="font-size:60%">
    <tr><td colspan=2><center><code><b>dense</b> 2 4 ⦶ (<b>dense</b> 4 2 <span style="font-size: 110%">⊖</span> <b>dense</b> 4 2)</code></center></td></tr>
    <tr style="background-color:white">
        <td style="width:50%">
<pre>let x0 = dense (2, 4)
</pre>
</td><td>
<pre>p1 = Population(2, ...)
p3 = Population(4, ...)
layer0 = Dense(p1, p3)
</pre>
</td></tr>
<tr style="background-color:white"><td style="width:50%">
<pre>let x1 = replicate 4
let x2 = connect_layers x0 x1
</pre>
</td><td>
<pre>
p5 = Population(4, ...)
p9 = Population(4, ...)
layer3 = Replicate(p3, (p5, p9))
</pre>
</td></tr>    
    <tr style="background-color:white"><td>
<pre>let x3 = dense (4, 2) 
let x4 = dense (4, 2) 
let x5 = connect_parallel x3 x4
let x6 = connect_layers x2 x5
</pre>
</td><td>
<pre>
p7 = Population(2, ...)
p11 = Population(2, ...)
layer1 = Dense(p5, p7)
layer2 = Dense(p9, p11)
</pre>
</td></tr><tr style="background-color:white"><td>
<pre>let x7 = merge (2, 2) 
let x8 = connect_layers x6 x7
</pre>
</td><td>
<pre>p13 = Population(4, ...)
layer4 = Merge((p7, p11), p13)
</pre>
</td>
    </tr>
</table>

## 4. Classification experiments

1. Experimental setup
2. Experimental results
3. Conclusions

### 4.1 Experimental setup

* ANN: Futhark on OpenCL
* SNN: NEST via PyNN

* Fixed neuron parameters, excitatory weights
* 10 experiments with 80/20% training/testing split

* SNN experiments run twice: 
  * Random weights
  * Weight transfer from Futhark to NEST

### 4.1 Experimental results

Accuracies with standard deviations

| Task | ANN | SNN w/ random weights | SNN w/ transferred weights | Chance level |
| --- | --- | --- | --- | --- |
| NAND | 1.000 | 0.370 $\pm$ 0.040 | 0.69 $\pm$ 0.216 | 0.75 |
| XOR | 1.000 | 0.530 $\pm$ 0.038 | 0.585 $\pm$ 0.033 | 0.5 |
| Sequential MNIST | 0.710 | 0.147 $\pm$ 0.044 | 0.098 $\pm$ 0.006 | 0.1 |



### 4.2 NAND and XOR: Learning

<img src="../report/images/nand.png" style="width:47%; float:left; margin: 0;"/>
<img src="../report/images/xor.png" style="width:47%; float: left; margin: 0;"/>

### 4.2 MNIST: Learning


<center><img src="../report/images/mnist_snn.png" style="width:70%"/></center>

### 4.3 Conclusions

* Bad prediction scores
  * Imprecise transfer model
  * Solely excitatory connections
  * No exploration of hyperparameters
* Learning occurs
* Improvement when importing ANN weights 
* Large number of dead neurons
  * ReLU model non-differentiable around 0

## 5. Findings and closing remarks
  1. Thesis findings
  2. Thesis contributions
  3. Learning objectives
  4. Reflection

### 5.1 Thesis findings

<table><tr>
    <th>Hypothesis</th><th>Question</th><th>Result</th></tr>
    <tr><td rowspan="2">The Volr DSL can translate into spiking and non-spiking neural networks such that the network topologies are retained</td><td>Translation to ANN</td><td>Confirmed</td>
    </tr>
    <tr><td>Translation to SNN</td><td>Confirmed</td></tr>
    <tr>
    <td rowspan="2">Using training it is possible for spiking and non-spiking models to solve an MNIST recognition task</td><td>Learning an MNIST task in ANN</td><td>Confirmed</td>
    </tr>
    <tr><td>Learning an MNIST task in SNN</td><td>Unconfirmed</td></tr>
    </table>

### 5.2 Contributions

Contributions:
 1. a domain-specific language for modelling neural network topologies
 2. a preliminary model for generalisable learning in SNN
 3. a method for transferring optimised non-spiking parameters to SNN

### 5.3 Learning objectives

* Survey models for learning through backpropagation in spiking and non-spiking neural networks
* Implement backpropagation learning in spiking and non-spiking neural networks
* Implement ANNs in the data-parallel functional language Futhark
* Implement SNNs in NEST and BrainScaleS
* Analyse and compare the SNN model performance through learning rate and accuracy scores

### 5.4 Reflection

* Perform *early* surveys of frameworks
* Work needed to explore translation from ANN to SNN
  * Excitatory and inhibitory connections
* Reuse code
  * Optimisers
  * Deep learning frameworks

## Modelling learning systems in artificial and spiking neural networks

</br></br>

* Author: Jens Egholm Pedersen ``<xtp778@alumni.ku.dk>``
* Supervisor: Martin Elsman ``<mael@diku.dk>``